In [1]:
#import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


In [2]:
#create the set of random longitudes and latitudes using the np method with the ranom uniform function practice in random_numbers.ipynb
lats = np.random.uniform(-90.000, 90.000, size= 1500)
lngs = np.random.uniform(-180.000, 180.000, size = 1500)

#pack the lats and longs together by zipping them with the zip() function
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
#add the latitudes and longitudes to a list (they have been stored as a zip object)
#you can only unzip a zipped tuple once before it's removed from memory
#so make sure you unzip it into a list right away

coordinates = list(lat_lngs)

In [4]:
#We need to use the citypy module to access the closest cities to given coordinate
#Went to anaconda prompt(python data) installed citpy with "pip install citipy"
#https://github.com/wingchen/citipy for inf0
from citipy import citipy


In [5]:
#create a list for holding cities

cities = []

#identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #if the city is unique, add it to the cities list
    if city not in cities:
        cities.append(city)
#print the city count to confirm a sufficent list
len(cities)

611

In [6]:
import requests

In [7]:

#import the API key
from config import weather_api_key


In [8]:
#Starting URL for Weather API Call from https://openweathermap.org/current#name
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)


http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=__redacted__


In [9]:
#create an endpoint URL for a city to specify
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=__redacted__&q=Boston


In [10]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather
#create output for request errors in case the url doesnt work and doesnt produce a status of 200
if city_weather.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")

City Weather found.


In [11]:
#get the text of the 'get'request
city_weather.text

'{"coord":{"lon":-71.0598,"lat":42.3584},"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02n"}],"base":"stations","main":{"temp":44.73,"feels_like":41.54,"temp_min":42.8,"temp_max":46.99,"pressure":1021,"humidity":81},"visibility":10000,"wind":{"speed":5.75,"deg":170},"clouds":{"all":20},"dt":1617940326,"sys":{"type":1,"id":3486,"country":"US","sunrise":1617876851,"sunset":1617923844},"timezone":-14400,"id":4930956,"name":"Boston","cod":200}'

In [12]:
#get the JSON text of the 'get' request, much easier to read and save to a variable
boston_data = city_weather.json()
boston_data

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 44.73,
  'feels_like': 41.54,
  'temp_min': 42.8,
  'temp_max': 46.99,
  'pressure': 1021,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 5.75, 'deg': 170},
 'clouds': {'all': 20},
 'dt': 1617940326,
 'sys': {'type': 1,
  'id': 3486,
  'country': 'US',
  'sunrise': 1617876851,
  'sunset': 1617923844},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [13]:
#use the 'sys' key from the json text, we see above that the country ID is
#in this key and we can pull that by using the "country" key after
boston_data["sys"]["country"]

'US'

In [14]:
#get all interesting data and save it to variables
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

42.3584 -71.0598 46.99 81 20 5.75


In [15]:
#pull the date timestamp from the data
boston_data["dt"]

1617940326

In [16]:
#convert that timestamp into ISO, a standardzed date format
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date)

datetime.datetime(2021, 4, 9, 3, 52, 6)

In [17]:
#convert the previously acquired ISO date into a readable format using
#the method strftime() that lets you add how you want the string to look
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-04-09 03:52:06'

In [18]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters that start at 1 not 0.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [19]:
#create a loop through our generated city list to build a url for each city

#for i in range(len(cities)):

#use enumerate() to retrieve both the citie index and city name in the same for loop
for i, city in enumerate(cities):
    #group the cities in dets of 50 for logging purposes
    if (i % 50 == 0 and i > 50):
        set_count +=1
        record_count = 1
    #creat and endpoint url with each city
    city_url = url + "&q=" + cities[i]  
    
    #log the url, record and set numbers and the city name
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    #add 1 to the record count
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Cloudiness": city_clouds,
                          "Country": city_country, 
                          "Date": city_date,  
                          "Humidity": city_humidity,                          
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Wind Speed": city_wind})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | mataura
Processing Record 2 of Set 1 | lebu
Processing Record 3 of Set 1 | ixtapa
Processing Record 4 of Set 1 | atuona
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 7 of Set 1 | bluff
Processing Record 8 of Set 1 | geraldton
Processing Record 9 of Set 1 | mrirt
City not found. Skipping...
Processing Record 10 of Set 1 | mahebourg
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | amderma
City not found. Skipping...
Processing Record 13 of Set 1 | raga
Processing Record 14 of Set 1 | makakilo city
Processing Record 15 of Set 1 | albany
Processing Record 16 of Set 1 | airai
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | saint-philippe
Processing Record 19 of Set 1 | wagar
Processing Record 20 of Set 1 | jamestown
Processing Record 21 of Set 1 | cherskiy
Processing Record 22 of Set 1 | laurentides
Processing Record 23 of Set 1 | 

Processing Record 42 of Set 3 | bengkulu
Processing Record 43 of Set 3 | ilulissat
Processing Record 44 of Set 3 | bredasdorp
Processing Record 45 of Set 3 | vestmanna
Processing Record 46 of Set 3 | bambous virieux
Processing Record 47 of Set 3 | kulhudhuffushi
Processing Record 48 of Set 3 | formoso do araguaia
City not found. Skipping...
Processing Record 49 of Set 3 | pevek
Processing Record 50 of Set 3 | grand gaube
Processing Record 1 of Set 4 | castro
Processing Record 2 of Set 4 | esperance
Processing Record 3 of Set 4 | paradise
Processing Record 4 of Set 4 | glendive
Processing Record 5 of Set 4 | hovd
Processing Record 6 of Set 4 | samarai
Processing Record 7 of Set 4 | qianan
Processing Record 8 of Set 4 | kuche
City not found. Skipping...
Processing Record 9 of Set 4 | huarmey
Processing Record 10 of Set 4 | arraial do cabo
Processing Record 11 of Set 4 | alyangula
Processing Record 12 of Set 4 | khorixas
Processing Record 13 of Set 4 | ambilobe
Processing Record 14 of Set

Processing Record 32 of Set 7 | coihaique
Processing Record 33 of Set 7 | chara
Processing Record 34 of Set 7 | temaraia
City not found. Skipping...
Processing Record 35 of Set 7 | singapore
Processing Record 36 of Set 7 | tiksi
Processing Record 37 of Set 7 | mastic beach
Processing Record 38 of Set 7 | adrar
Processing Record 39 of Set 7 | nuuk
Processing Record 40 of Set 7 | progreso
Processing Record 41 of Set 7 | chapada dos guimaraes
Processing Record 42 of Set 7 | leningradskiy
Processing Record 43 of Set 7 | perth
Processing Record 44 of Set 7 | ust-ishim
Processing Record 45 of Set 7 | calbe
Processing Record 46 of Set 7 | redcar
Processing Record 47 of Set 7 | kentau
Processing Record 48 of Set 7 | manyana
Processing Record 49 of Set 7 | katima mulilo
Processing Record 50 of Set 7 | sucua
Processing Record 1 of Set 8 | la uribe
City not found. Skipping...
Processing Record 2 of Set 8 | hun
Processing Record 3 of Set 8 | bantry
Processing Record 4 of Set 8 | roald
Processing R

Processing Record 20 of Set 11 | bitam
Processing Record 21 of Set 11 | soligalich
Processing Record 22 of Set 11 | haibowan
City not found. Skipping...
Processing Record 23 of Set 11 | te anau
Processing Record 24 of Set 11 | maneadero
Processing Record 25 of Set 11 | neon karlovasion
Processing Record 26 of Set 11 | biograd na moru
Processing Record 27 of Set 11 | nhulunbuy
Processing Record 28 of Set 11 | pacific grove
Processing Record 29 of Set 11 | chikmagalur
Processing Record 30 of Set 11 | kuznechikha
Processing Record 31 of Set 11 | hokitika
Processing Record 32 of Set 11 | waiuku
Processing Record 33 of Set 11 | xitla
City not found. Skipping...
Processing Record 34 of Set 11 | lasa
Processing Record 35 of Set 11 | montepuez
Processing Record 36 of Set 11 | phan thiet
Processing Record 37 of Set 11 | buan
Processing Record 38 of Set 11 | gilbues
Processing Record 39 of Set 11 | eyl
Processing Record 40 of Set 11 | bara
Processing Record 41 of Set 11 | trinidad
Processing Rec

In [20]:
len(city_data)

562

In [21]:
#convert this array of dictionaries with city data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mataura,0,NZ,2021-04-09 03:44:03,41,-46.1927,168.8643,66.67,1.25
1,Lebu,36,CL,2021-04-09 03:44:24,89,-37.6167,-73.6500,52.02,3.47
2,Ixtapa,1,MX,2021-04-09 03:45:56,83,20.7000,-105.2000,75.20,2.82
3,Atuona,0,PF,2021-04-09 03:44:08,74,-9.8000,-139.0333,81.23,16.11
4,Ushuaia,20,AR,2021-04-09 03:43:22,61,-54.8000,-68.3000,42.80,11.50
5,Bluff,0,NZ,2021-04-09 03:44:05,59,-46.6000,168.3333,58.26,2.06
6,Geraldton,97,AU,2021-04-09 03:44:35,37,-28.7667,114.6000,84.20,18.41
7,Mahebourg,40,MU,2021-04-09 03:44:33,78,-20.4081,57.7000,78.80,8.05
8,Rikitea,98,PF,2021-04-09 03:43:58,67,-23.1203,-134.9692,77.02,6.98
9,Raga,67,SS,2021-04-09 03:52:08,56,8.4596,25.6780,77.11,3.09


In [24]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]

In [25]:
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Mataura,NZ,2021-04-09 03:44:03,-46.1927,168.8643,66.67,41,0,1.25
1,Lebu,CL,2021-04-09 03:44:24,-37.6167,-73.6500,52.02,89,36,3.47
2,Ixtapa,MX,2021-04-09 03:45:56,20.7000,-105.2000,75.20,83,1,2.82
3,Atuona,PF,2021-04-09 03:44:08,-9.8000,-139.0333,81.23,74,0,16.11
4,Ushuaia,AR,2021-04-09 03:43:22,-54.8000,-68.3000,42.80,61,20,11.50
5,Bluff,NZ,2021-04-09 03:44:05,-46.6000,168.3333,58.26,59,0,2.06
6,Geraldton,AU,2021-04-09 03:44:35,-28.7667,114.6000,84.20,37,97,18.41
7,Mahebourg,MU,2021-04-09 03:44:33,-20.4081,57.7000,78.80,78,40,8.05
8,Rikitea,PF,2021-04-09 03:43:58,-23.1203,-134.9692,77.02,67,98,6.98
9,Raga,SS,2021-04-09 03:52:08,8.4596,25.6780,77.11,56,67,3.09


In [26]:
#create output file (CSV) to store data
output_data_file = "weather_data/cities.csv"

#Export the city_data into the csv with the index label (or column A) 
#header as "City_ID." If we ever need to export the CSV file to a 
#DataFrame, that header will be present in the DataFrame

city_data_df.to_csv(output_data_file, index_label = "City_ID")